In [ ]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import matplotlib
import collections
from operator import itemgetter
import csv

sns.set_style("whitegrid")
matplotlib.style.use("ggplot")

In [ ]:
df = pd.read_csv("/home/hduser/workspaces/datakind-workspace/coc.csv", low_memory=False)
print df.head()

In [ ]:
df.columns

In [ ]:
df["Complaint.Type"].value_counts().plot(kind="barh", figsize=(10, 25))

In [ ]:
df["Complaint.Date"].head()

# Convert date types 
dateFormat = "%m/%d/%Y %H:%M:%S"
df["Complaint.Date"] = pd.to_datetime(df["Complaint.Date"], format=dateFormat)
df["Resolution.Date"] = pd.to_datetime(df["Resolution.Date"], format=dateFormat)

# add additional columns from the date columns for easier processing
df["Complaint.Year"] = df["Complaint.Date"].apply(lambda _ : _.year)
df["Complaint.Month"] = df["Complaint.Date"].apply(lambda _ : _.month)
df["Complaint.Day"] = df["Complaint.Date"].apply(lambda _ : _.day)
df["Complaint.Hour"] = df["Complaint.Date"].apply(lambda _ : _.hour)

df["Resolution.Year"] = df["Resolution.Date"].apply(lambda _ : _.year)
df["Resolution.Month"] = df["Resolution.Date"].apply(lambda _ : _.month)
df["Resolution.Day"] = df["Resolution.Date"].apply(lambda _ : _.day)
df["Resolution.Hour"] = df["Resolution.Date"].apply(lambda _ : _.hour)

In [ ]:
from nltk.stem.porter import PorterStemmer
import nltk.data
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.corpus import stopwords
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
# The TreeBank word tokenizer
tokenizer = TreebankWordTokenizer()
# Porter Stemmer
stemmer = PorterStemmer()
# common english stop words
stop_words = set(stopwords.words('english')).union([stemmer.stem(sw) for sw in stopwords.words('english')])

def tokenize(text):
    sentences = sent_detector.tokenize(text.lower())
    tokens = []
    for sentence in sentences:
        # tokenize
        toks = tokenizer.tokenize(sentence)
        # strip and stem the words
        tokens.extend(map(lambda x: stemmer.stem(x.strip()), toks))    
    # filter out non-alphanumeric words i.e punctuation and remove stop words    
    return filter(lambda x: x.isalnum() and x not in stop_words, tokens)


def tokenizeColumn(df, inputColumnName, outputColumnName):
    # to deal with python's damned $*@&# unicode issues
    df.tempColumn = df[inputColumnName].apply(lambda _: unicode(str(_), "utf-8"))
    df[outputColumnName] = df.tempColumn.apply(tokenize)
    

In [ ]:
smallDf = df.head(1000)
tokenizeColumn(smallDf, "Complaint.Details", "Complaint.DetailsTokenized")

In [ ]:
smallDf["Complaint.DetailsTokenized"].head()

In [ ]:
def computeJaccardIndex(df, idColumn, columnName):
    jaccard = {}
    for i, row1 in zip(df[idColumn], df[columnName]):
        jaccard[i] = {}
        for j, row2 in zip(df[idColumn], df[columnName]):            
            v1 = set(row1)
            v2 = set(row2)
            if len(v1) == 0 or len(v2) == 0:
                continue
            jaccard[i][j] = float(len(v1.intersection(v2))) / len(v1.union(v2))
    return jaccard
# this takes a long time
#jaccard = computeJaccardIndex(smallDf, "Unnamed: 0", "Complaint.DetailsTokenized")
jaccard = {}

In [ ]:
done = set()
writer =  open("first100Results.csv", "w")
for id1, jacDict in jaccard.iteritems():
    for id2, value in jaccard[id1].iteritems():
        idSet = "_".join(sorted([str(id1), str(id2)]))        
        if value > 0.80 and id1 != id2 and idSet not in done:            
            done.add(idSet)    
            writer.write("{},{},{},\"{}\",\"{}\"\n".format(value, id1, id2,
                   smallDf[smallDf["Unnamed: 0"] == id1]["Complaint.Details"].values[0].replace("\n", "\\n"), 
                   smallDf[smallDf["Unnamed: 0"] == id2]["Complaint.Details"].values[0].replace("\n", "\\n")))
writer.close()
            

In [ ]:
df["Area"].value_counts().plot(kind="barh", figsize =(10, 45))

In [ ]:
print np.unique(df["Locality"]).shape
print "nan" in np.unique(df["Locality"])

In [ ]:
print np.unique(df["Street"]).shape
print "nan" in np.unique(df["Street"])

In [ ]:
df["Region"].value_counts().plot(kind="bar")

In [ ]:
df["Zone"].value_counts().plot(kind="bar", figsize=(15, 10))

In [ ]:
df["Zone"].value_counts().plot(kind="barh", figsize = (15, 10))

In [ ]:
df["Ward"].value_counts().plot(kind = "barh", figsize = (15, 45))

In [ ]:
df["Department"].value_counts().plot(kind="barh", figsize=(15, 20))

In [ ]:
df["Complaint.Details"].value_counts()[:5].plot(kind="bar", figsize = (15, 10))

In [ ]:
df["Email"].value_counts()[:10]

In [ ]:
df["Complaint.Year"].value_counts().plot(kind="bar", figsize=(15,10))

In [ ]:
complaintType = "Dog menace "
year = 2014
c = collections.Counter(df[(df["Complaint.Type"] == complaintType) & (df["Complaint.Year"] == year)]["Complaint.Month"])
sns.barplot(np.array(map(itemgetter(0), c.items())),
        np.array(map(itemgetter(1), c.items())))

In [ ]:
def plotComplaintType(df):
    for complaintType in df["Complaint.Type"].unique():
        for year in df["Complaint.Year"].unique():
            slices = df[(df["Complaint.Type"] == complaintType) & (df["Complaint.Year"] == year)]
            if len(slices) == 0:
                continue
            c = collections.Counter(slices["Complaint.Month"])
            sns.barplot(np.array(map(itemgetter(0), c.items())),
                np.array(map(itemgetter(1), c.items())))
            plt.title("Complaint Type: {}, Year: {}".format(complaintType, year))
            plt.xlabel("Month")
            plt.ylabel("Number of Complaints")
            plt.savefig("{}_{}.png".format(year, complaintType.replace("/", "")), dpi=200)
            plt.clf()

In [ ]:
def plotComplaintType1(df):
    for complaintType in df["Complaint.Type"].unique():    
        counts = collections.OrderedDict()
        for year in sorted(df["Complaint.Year"].unique()):        
                slices = df[(df["Complaint.Type"] == complaintType) & (df["Complaint.Year"] == year)]
                if len(slices) == 0:
                    continue
                c = collections.Counter(slices["Complaint.Month"])        
                for month in range(1, 13):
                    counts["{},{}".format(year, month)] = c.get(month, 0)
        plt.figure(figsize=(15, 6))
        pl = sns.barplot(np.array(map(itemgetter(0), counts.items())),
                    np.array(map(itemgetter(1), counts.items())), palette="Blues")
        locs, labels = plt.xticks()
        plt.setp(labels, rotation=90)
        plt.title("Complaint Type: {}".format(complaintType))
        plt.xlabel("Year / Month")
        plt.ylabel("Number of Complaints")
        plt.savefig("{} across years.png".format(complaintType.replace("/", "")), dpi=300)

In [ ]:
undup = df.drop_duplicates(['Complaint.Type', 'Area', 'Locality', 'Street', 'Region', 'Zone', 'Ward', 'Complaint.Details', u'Email', u'Mobile', u'Complaint.Month', "Complaint.Year", u'Complaint.Day'])

In [ ]:
print "Original: {}, De-duped: {}, Duplicates: {}".format(df.shape, undup.shape, df.shape[0] - undup.shape[0])

In [ ]:
def saveDf(df, fileName):
    undup.to_csv(fileName, na_rep="NA", index=False, encoding="utf-8", date_format=dateFormat, quoting=csv.QUOTE_ALL)
    
# saveDf(undup, "coc_unduplicated.csv")

In [ ]:
problemCategoryDictionary = df[df["Is.Resolved"] == "YES"].drop_duplicates(["Complaint.Type","Street", "Resolution.Year", "Resolution.Month", "Resolution.Day"])
print problemCategoryDictionary.shape
# saveDf(problemCategoryDictionary, "problemCategoryDictionary.csv")

In [ ]:
for year in sorted(df["Complaint.Year"].unique()):
    c = collections.Counter(df[df["Complaint.Year"] == year]["Is.Resolved"])
    print year, c
    print "Percentage Resolved: {}".format(float(c["YES"]) / (c["YES"] + c["NO"]))
    